<a href="https://colab.research.google.com/github/Renjian-buchai/buildingBloCS/blob/main/PillDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CSV Manipilation


In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('your_file.csv')

# Retain only the "filename" and "region_shape_attributes" columns
df = df[['filename', 'region_shape_attributes']]

# Save the modified DataFrame to a new CSV file
df.to_csv('new_file.csv', index=False)

In [ ]:
import json

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('your_file.csv')

# Extract x_min, y_min, x_max, and y_max from region_shape_attributes
df['region_shape_attributes'] = df['region_shape_attributes'].apply(lambda x: json.loads(x))
df['x_min'] = df['region_shape_attributes'].apply(lambda x: x['x'])
df['y_min'] = df['region_shape_attributes'].apply(lambda x: x['y'])
df['x_max'] = df['region_shape_attributes'].apply(lambda x: x['x'] + x['width'])
df['y_max'] = df['region_shape_attributes'].apply(lambda x: x['y'] + x['height'])

# Remove the region_shape_attributes column
df = df.drop(columns=['region_shape_attributes'])

# Save the modified DataFrame to a new CSV file
df.to_csv('new_file.csv', index=False)

Imports

In [ ]:
# import the necessary packages
# from pyimagesearch import config

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

Set some constants

In [ ]:
ANNOTS_PATH = "/content/Test/annotations.csv"
IMAGES_PATH = "/content/Test"

TEST_FILENAMES = "/content/output/test_images.txt"
PLOT_PATH = "/content/output/plot.png"
MODEL_PATH = "/content/output/detector.h5"

INIT_LR = 1e-4
NUM_EPOCHS = 25
BATCH_SIZE = 32

In [ ]:
rows = open(ANNOTS_PATH).read().strip().split("\n")


data = []
targets = []
filenames = []

CSV reading and image loading

In [ ]:
for row in rows:
	# break the row into the filename and bounding box coordinates
  row = row.split(",")
  (filename, startX, startY, endX, endY) = row
  imagePath = os.path.sep.join([IMAGES_PATH, filename])
  image = cv2.imread(imagePath)
  (h, w) = image.shape[:2]
  # scale the bounding box coordinates relative to the spatial
  # dimensions of the input image
  startX = float(startX) / w
  startY = float(startY) / h
  endX = float(endX) / w
  endY = float(endY) / h
    # load the image and preprocess it
  image = load_img(imagePath, target_size=(224, 224))
  image = img_to_array(image)
  # update our list of data, targets, and filenames
  data.append(image)
  targets.append((startX, startY, endX, endY))
  filenames.append(filename)

Partion Dataset

In [ ]:
# convert the data and targets to NumPy arrays, scaling the input
# pixel intensities from the range [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")
# partition the data into training and testing splits using 90% of
# the data for training and the remaining 10% for testing
split = train_test_split(data, targets, filenames, test_size=0.10,
	random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]
# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print("[INFO] saving testing filenames...")
f = open(TEST_FILENAMES, "w")
f.write("\n".join(testFilenames))
f.close()

VGG finetuning

In [ ]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)

# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(lr=INIT_LR)
model.compile(loss="mse", optimizer=opt)
print(model.summary())
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
H = model.fit(
	trainImages, trainTargets,
	validation_data=(testImages, testTargets),
	batch_size=BATCH_SIZE,
	epochs=NUM_EPOCHS,
	verbose=1)

Serialise and plot training history

In [ ]:
# serialize the model to disk
print("[INFO] saving object detector model...")
model.save(MODEL_PATH, save_format="h5")
# plot the model training history
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Bounding Box Regression Loss on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(PLOT_PATH)